# Understanding Stokes Flow

I want to know how the stokes equation is solved in underworld 3. The stokes equation is of the form 
$$\nabla \cdot \vec{\sigma} = \vec{f}$$

In [2]:
## import underworld
import underworld3 as uw

Lets solve our equation on the domain $[0,1]\times[0,1]$ and discretise within cells of size $\frac{1}{20}$ with a quadature degree 2.

In [11]:
mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0.0, 0.0), maxCoords=(1.0, 1.0), cellSize=1 / 20, qdegree=5
)

Lets print out our mesh to have a look at it

In [12]:
## print out what the mesh looks like
if uw.mpi.size == 1:

    # plot the mesh
    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 750]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True

    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")

    pl = pv.Plotter()

    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        use_transparency=False,
    )

    pl.show(cpos="xy")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_83049/1705306568.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


VTKRenderWindowSynchronized(vtkCocoaRenderWindow, height=750, min_width=750, sizing_mode='stretch_width')

Lets define two variables in the system, pressure $p$ and velocity $v$ on the mesh.

In [13]:
velocity = uw.discretisation.MeshVariable("r/u", mesh, 2, degree=3)
pressure = uw.discretisation.MeshVariable("r/p", mesh, 1, degree=3)

In [14]:
mesh.view()

Mesh 1
| Variable Name       | component | degree | type        |
| ------------------------------------------------------ |
| ru                  |    2      |   3    |  VECTOR   |
| rp                  |    1      |   3    |  SCALAR   |
| ------------------------------------------------------ |


In [15]:
stokes_solver = uw.systems.Stokes(mesh, velocityField=velocity, pressureField=pressure)

In [16]:
stokes_solver


This class provides functionality for a discrete representation
of the Stokes flow equations assuming an incompressibility
(or near-incompressibility) constraint.

$$\frac{\partial}{\partial x_j} \left( \frac{\eta}{2} \left[ \frac{\partial u_i}{\partial x_j}  +
        \frac{\partial u_j}{\partial x_i} \right]\right) - \frac{\partial p}{\partial x_i} = f_i$$

$$\frac{\partial u_i}{\partial x_i} = 0$$

## Properties

  - The viscosity, $ \eta $ is provided by setting the `constitutive_model` property to
one of the `uw.systems.constitutive_models` classes and populating the parameters.
It is usually a constant or a function of position / time and may also be non-linear
or anisotropic.

  - The bodyforce term, $ f_i $ is provided through the `bodyforce` property.

  - The Augmented Lagrangian approach to application of the incompressibility
constraint is to penalise incompressibility in the Stokes equation by adding
$ \lambda \nabla \cdot \mathbf{u} $ when the weak form of the equations is constructed.
(this is in addition to the constraint equation, unlike in the classical penalty method).
This is activated by setting the `penalty` property to a non-zero floating point value.

  - A preconditioner is usually required for the saddle point system and this is provided
though the `saddle_preconditioner` property. A common choice is $ 1/ \eta $ or
$ 1 / \eta + 1/ \lambda $ if a penalty is used


## Notes

  - The interpolation order of the `pressureField` variable is used to determine the integration order of
the mixed finite element method and is usually lower than the order of the `velocityField` variable.

  - It is possible to set discontinuous pressure variables by setting the `p_continous` option to `False`
(currently this is not implemented).

  - The `solver_name` parameter sets the namespace for PETSc options and should be unique and
compatible with the PETSc naming conventions.




This solver is formulated in 2 dimensions

In [ ]:
stokes_solver.constitutive_model = uw.systems.constitutive_models.DiffusionModel.